In [1]:
# Initialize Otter
import otter
grader = otter.Notebook()

# Lab 4: Data Cleaning and Visualization

In this lab you will be working on visualizing a dataset from the City of Berkeley containing data on calls to the Berkeley Police Department. Information about the dataset can be found [at this link](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5).

Note: This lab will not work on older versions of Python; make sure to work on DataHub.

**This assignment should be completed and submitted by 11:59 PM on Thursday July 2, 2020.**

**Warning: This lab includes an analysis of crime in Berkeley. If you feel uncomfortable about the topic, please feel free to contact your GSI or the instructors.**

## Setup

Note that we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [2]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 9)

## Part 1: Cleaning and Exploring the Data

To retrieve the dataset, we will use the `ds100_utils.fetch_and_cache` utility.

In [3]:
import ds100_utils

data_dir = 'data'
data_url = 'http://www.ds100.org/su20/resources/assets/datasets/lab04_data_su20.zip'
file_name = 'lab04_data_su20.zip'

dest_path = ds100_utils.fetch_and_cache(data_url=data_url, file=file_name, data_dir=data_dir)
print(f'Located at {dest_path}')

Downloading... Done!
Located at data/lab04_data_su20.zip


We will now directly unzip the ZIP archive and start working with the uncompressed files.

Note: There is no single right answer regarding whether to work with compressed files in their compressed state or to uncompress them on disk permanently. If you for example need to work with multiple tools on the same files, or write many notebooks to analyze them, and they are not too large, it may be more convenient to uncompress them once.  But you may also have situations where you find it preferable to work with the compressed data directly.  

Python gives you tools for both approaches, and you should know how to perform both tasks in order to choose the one that best suits the problem at hand.

---

Run the cell below to extract the zip file into the data directory.

In [4]:
my_zip = zipfile.ZipFile(dest_path, 'r')
my_zip.extractall(data_dir)

Now, we'll use a method of the `Pathlib.Path` class called `glob` to list all files in the `data` directory. You will find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).

Below, we use pathlib's `glob` method to store the list of all files' names from the `data_dir` directory in the variable `file_names`. These names should be strings that contain only the file name (e.g. `dummy.txt` not `data/dummy.txt`). The asterisk (*) character is used with the `glob` method to match any string.

In [5]:
from pathlib import Path
data_dir_path = Path('data') # creates a Path object that points to the data directory
file_names = [x.name for x in data_dir_path.glob('*') if x.is_file()]
file_names

['dummy.txt',
 'hello_world.py',
 'Berkeley_PD_-_Calls_for_Service.csv',
 'lab04_data_su20.zip',
 'do_not_readme.md',
 'ben_kurtovic.py']

Let's now load the CSV file we have into a `pandas.DataFrame` object.

In [6]:
calls = pd.read_csv("data/Berkeley_PD_-_Calls_for_Service.csv")
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,18022300,DISTURBANCE,04/18/2018 12:00:00 AM,22:17,DISORDERLY CONDUCT,3,09/06/2018 03:30:12 AM,"OREGON STREET &amp; MCGEE AVE\nBerkeley, CA\n(...",OREGON STREET & MCGEE AVE,Berkeley,CA
1,18026683,THEFT MISD. (UNDER $950),05/09/2018 12:00:00 AM,21:25,LARCENY,3,09/06/2018 03:30:13 AM,"200 UNIVERSITY AVE\nBerkeley, CA\n(37.865511, ...",200 UNIVERSITY AVE,Berkeley,CA
2,18038550,THEFT MISD. (UNDER $950),05/18/2018 12:00:00 AM,20:00,LARCENY,5,09/06/2018 03:30:09 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA
3,18014810,BURGLARY AUTO,03/13/2018 12:00:00 AM,08:50,BURGLARY - VEHICLE,2,09/06/2018 03:30:08 AM,"1200 SIXTH ST\nBerkeley, CA\n(37.881142, -122....",1200 SIXTH ST,Berkeley,CA
4,18018643,ALCOHOL OFFENSE,03/31/2018 12:00:00 AM,13:29,LIQUOR LAW VIOLATION,6,09/06/2018 03:30:11 AM,"CENTER STREET &amp; SHATTUCK AVE\nBerkeley, CA...",CENTER STREET & SHATTUCK AVE,Berkeley,CA


We see that the fields include a case number, the offense type, the date and time of the offense, the "CVLEGEND" which appears to be related to the offense type, a "CVDOW" which has no apparent meaning, a date added to the database, and the location spread across four fields.

Let's also check some basic information about these files using the `DataFrame.info` and `DataFrame.describe` methods.

In [10]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3788 entries, 0 to 3787
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CASENO          3788 non-null   int64 
 1   OFFENSE         3788 non-null   object
 2   EVENTDT         3788 non-null   object
 3   EVENTTM         3788 non-null   object
 4   CVLEGEND        3788 non-null   object
 5   CVDOW           3788 non-null   int64 
 6   InDbDate        3788 non-null   object
 7   Block_Location  3788 non-null   object
 8   BLKADDR         3766 non-null   object
 9   City            3788 non-null   object
 10  State           3788 non-null   object
dtypes: int64(2), object(9)
memory usage: 325.7+ KB


In [11]:
calls.describe()

,CASENO,CVDOW
count,3.788000e+03,3788.000000
mean,1.804387e+07,2.997888
std,2.665970e+04,1.952160
min,1.801375e+07,0.000000
25%,1.802444e+07,1.000000
50%,1.803530e+07,3.000000
75%,1.804537e+07,5.000000
max,1.809135e+07,6.000000


Notice that the functions above reveal type information for the columns, as well as some basic statistics about the numerical columns found in the DataFrame. However, we still need more information about what each column represents. Let's explore the data further in Question 1.

Before we go over the fields to see their meanings, the cell below will verify that all the events happened in Berkeley by grouping on the `City` and `State` columns. You should see that all of our data falls into one group.

In [12]:
calls.groupby(["City","State"]).count()

,,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR
City,State,,,,,,,,,
Berkeley,CA,3788,3788,3788,3788,3788,3788,3788,3788,3766


### Question 1
Above, when we called `head`, it seemed like `OFFENSE` and `CVLEGEND` both contained information about the type of event reported. What is the difference in meaning between the two columns? One way to probe this is to look at the `value_counts` for each Series.

In [13]:
calls['OFFENSE'].value_counts().head(10)

BURGLARY AUTO               658
THEFT MISD. (UNDER $950)    633
DISTURBANCE                 277
VANDALISM                   248
VEHICLE STOLEN              235
THEFT FELONY (OVER $950)    214
BURGLARY RESIDENTIAL        194
ASSAULT/BATTERY MISD.       190
ROBBERY                     168
NARCOTICS                   118
Name: OFFENSE, dtype: int64

In [14]:
calls['CVLEGEND'].value_counts().head(10)

LARCENY                   871
BURGLARY - VEHICLE        658
DISORDERLY CONDUCT        279
ASSAULT                   263
VANDALISM                 248
MOTOR VEHICLE THEFT       235
FRAUD                     208
BURGLARY - RESIDENTIAL    194
ROBBERY                   168
DRUG VIOLATION            118
Name: CVLEGEND, dtype: int64

#### Question 1a

Above, it seems like `OFFENSE` is more specific than `CVLEGEND`, e.g. "LARCENY" vs. "THEFT FELONY (OVER $950)". If you're unfamiliar with the term, "larceny" is a legal term for theft of personal property.

To get a sense of how many subcategories there are for each `OFFENSE`, set `calls_by_cvlegend_and_offense` equal to a multi-indexed series where the data is first indexed on the `CVLEGEND` and then on the `OFFENSE`, and the data is equal to the number of offenses in the database that match the respective `CVLEGEND` and `OFFENSE`. For example, calls_by_cvlegend_and_offense["LARCENY", "THEFT FROM PERSON"] should return 24.

<!--
BEGIN QUESTION
name: q1a
-->

In [23]:
calls_by_cvlegend_and_offense = calls.groupby(['CVLEGEND', 'OFFENSE']).size()
calls_by_cvlegend_and_offense

CVLEGEND                OFFENSE                 
ALL OTHER OFFENSES      MUNICIPAL CODE                3
ARSON                   ARSON                        18
ASSAULT                 ASSAULT/BATTERY FEL.         73
                        ASSAULT/BATTERY MISD.       190
BURGLARY - COMMERCIAL   BURGLARY COMMERCIAL         112
BURGLARY - RESIDENTIAL  BURGLARY RESIDENTIAL        194
BURGLARY - VEHICLE      BURGLARY AUTO               658
DISORDERLY CONDUCT      DISTURBANCE                 277
                        VICE                          2
DRUG VIOLATION          NARCOTICS                   118
FAMILY OFFENSE          DOMESTIC VIOLENCE           108
FRAUD                   FRAUD/FORGERY                95
                        IDENTITY THEFT              113
HOMICIDE                HOMICIDE                      1
KIDNAPPING              KIDNAPPING                    6
LARCENY                 THEFT FELONY (OVER $950)    214
                        THEFT FROM PERSON            24

In [21]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b

In the cell below, set `answer1b` equal to a list of strings corresponding to the possible values for `OFFENSE` when `CVLEGEND` is "LARCENY". You can type the answer manually, or you can create an expression that automatically extracts the names.

<!--
BEGIN QUESTION
name: q1b
-->

In [40]:
answer1b = calls_by_cvlegend_and_offense["LARCENY"].index.tolist()
answer1b

['THEFT FELONY (OVER $950)', 'THEFT FROM PERSON', 'THEFT MISD. (UNDER $950)']

In [41]:
grader.check("q1b")

q1b results: All test cases passed!

## Question 2

What are the five crime types of CVLEGEND that have the most crime events? You may need to use `value_counts` to find the answer.
Save your results into `answer2` as a list of strings.

**Hint:** *The `keys` method of the Series class might be useful.*

<!--
BEGIN QUESTION
name: q2
-->

In [57]:
answer2 = calls['CVLEGEND'].value_counts().head(5).keys().tolist()
answer2

['LARCENY', 'BURGLARY - VEHICLE', 'DISORDERLY CONDUCT', 'ASSAULT', 'VANDALISM']

In [58]:
grader.check("q2")head

SyntaxError: invalid syntax (<ipython-input-58-733e25b75555>, line 1)

In [59]:
grader.check("q2")

q2 results: All test cases passed!

--- 

## Part 2: Visualization


## Pandas Examples

Pandas offers basic functionality for plotting. For example, the `DataFrame` and `Series` classes both have a `plot` method. 

As you learn to do data visualization, you may find the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)  helpful!

As an example of the built-in plotting functionality of pandas, the following example uses `plot` method of the `Series` class to generate a `barh` plot type to visually display the value counts for `CVLEGEND`.

There are also many other plots that we will explore throughout the lab.

In [60]:
ax = calls['CVLEGEND'].value_counts().plot(kind='barh')
ax.set_ylabel("Crime Category")
ax.set_xlabel("Number of Calls")
ax.set_title("Number of Calls By Crime Type");



## An Additional Note on Plotting in Jupyter Notebooks

You may have noticed that many of our code cells involving plotting end with a semicolon (;). This prevents any extra output from the last line of the cell that we may not want to see. Try adding this to your own code in the following questions!

### Question 3

Now it is your turn to make some plots using `pandas`  Let's start by transforming the data so that it is easier to work with We then will look at some distributions of the data. 

The CVDOW field isn't named helpfully and it is hard to see the meaning from the data alone. According to the website linked at the top of this notebook, CVDOW is actually indicating the day that events happened. 0->Sunday, 1->Monday ... 6->Saturday. 

#### Question 3a

Add a new column `Day` into the `calls` dataframe that has the string weekday (eg. 'Sunday') for the corresponding value in CVDOW. For example, if the first 3 values of `CVDOW` are `[3, 6, 0]`, then the first 3 values of the `Day` column should be `["Wednesday", "Saturday", "Sunday"]`.

**Hint:** *Try using the [Series.map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function on `calls["CVDOW"]`.  Can you assign this to the new column `calls["Day"]`?*

<!--
BEGIN QUESTION
name: q3a
-->

In [66]:
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
day_indices = range(7)
indices_to_days_dict = dict(zip(day_indices, days)) # Should look like {0:"Sunday", 1:"Monday", ..., 6:"Saturday"}
calls['Day'] = calls['CVDOW'].map(indices_to_days_dict)
calls

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day
0,18022300,DISTURBANCE,04/18/2018 12:00:00 AM,22:17,DISORDERLY CONDUCT,3,09/06/2018 03:30:12 AM,"OREGON STREET &amp; MCGEE AVE\nBerkeley, CA\n(...",OREGON STREET & MCGEE AVE,Berkeley,CA,Wednesday
1,18026683,THEFT MISD. (UNDER $950),05/09/2018 12:00:00 AM,21:25,LARCENY,3,09/06/2018 03:30:13 AM,"200 UNIVERSITY AVE\nBerkeley, CA\n(37.865511, ...",200 UNIVERSITY AVE,Berkeley,CA,Wednesday
2,18038550,THEFT MISD. (UNDER $950),05/18/2018 12:00:00 AM,20:00,LARCENY,5,09/06/2018 03:30:09 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Friday
3,18014810,BURGLARY AUTO,03/13/2018 12:00:00 AM,08:50,BURGLARY - VEHICLE,2,09/06/2018 03:30:08 AM,"1200 SIXTH ST\nBerkeley, CA\n(37.881142, -122....",1200 SIXTH ST,Berkeley,CA,Tuesday
4,18018643,ALCOHOL OFFENSE,03/31/2018 12:00:00 AM,13:29,LIQUOR LAW VIOLATION,6,09/06/2018 03:30:11 AM,"CENTER STREET &amp; SHATTUCK AVE\nBerkeley, CA...",CENTER STREET & SHATTUCK AVE,Berkeley,CA,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...
3783,18045829,THEFT MISD. (UNDER $950),08/15/2018 12:00:00 AM,08:42,LARCENY,3,09/06/2018 03:30:10 AM,"2300 TELEGRAPH AVE\nBerkeley, CA\n(37.868714, ...",2300 TELEGRAPH AVE,Berkeley,CA,Wednesday
3784,18040137,DISTURBANCE,07/17/2018 12:00:00 AM,10:34,DISORDERLY CONDUCT,2,09/06/2018 03:30:13 AM,"1100 UNIVERSITY AVE\nBerkeley, CA\n(37.869067,...",1100 UNIVERSITY AVE,Berkeley,CA,Tuesday
3785,18090816,VANDALISM,05/16/2018 12:00:00 AM,20:00,VANDALISM,3,09/06/2018 03:30:13 AM,"800 VICENTE RD\nBerkeley, CA\n",800 VICENTE RD,Berkeley,CA,Wednesday
3786,18024397,SEXUAL ASSAULT FEL.,04/28/2018 12:00:00 AM,17:00,SEX CRIME,6,09/06/2018 03:30:12 AM,"2700 BANCROFT WAY\nBerkeley, CA\n(37.869312, -...",2700 BANCROFT WAY,Berkeley,CA,Saturday


In [67]:
grader.check("q3a")

q3a results: All test cases passed!

#### Question 3b

Now let's look at the EVENTTM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`.


**Hint:** *Your code should only require one line*

<!--
BEGIN QUESTION
name: q3b
-->

In [81]:
calls['Hour'] = calls['EVENTTM'].str[0:2].astype('int64')
calls['Hour']

0       22
1       21
2       20
3        8
4       13
        ..
3783     8
3784    10
3785    20
3786    17
3787    21
Name: Hour, Length: 3788, dtype: int64

In [82]:
grader.check("q3b")

q3b results: All test cases passed!

#### Question 3c

Using `pandas`, Construct a vertical bar plot with the count of the number of calls (entries in the table) for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). Be sure that your axes are labeled and that your plot is titled.  There should be approximately 500 calls per day.

**Hint:** *Given a series `s`, and an array `coolIndex` that has the same entries as in `s.index`, `s[coolIndex]` will return a copy of the series in the same order as `coolIndex`.*

<!--
BEGIN QUESTION
name: q3c
-->

In [90]:
ax_3c = calls['Day'].value_counts()[days].plot(kind='bar')
ax_3c.set_title("Number of Call for Each Day of the Week")
ax_3c.set_ylabel("Number of Call")

# Leave this for grading purposes
ax_3c = plt.gca()

In [91]:
grader.check("q3c")

q3c results: All test cases passed!

#### Question 3d


Using `pandas`, construct a line plot with the count of the number of calls (entries in the table) for each hour of the day  **ordered by the time** (eg. `12:00 AM`, `1:00 AM`, ...). Please use the provided variable `hours` in your answer. Be sure that your axes are labeled and that your plot is titled. 

<!--
BEGIN QUESTION
name: q3d

-->

In [97]:
hours = list(range(24))
ax_3d = calls['Hour'].value_counts()[hours].plot()
ax_3d.set_title("Number of the Calls for Each Hour of the Day")
ax_3d.set_ylabel("Number of Call")
ax_3d.set_xlabel("Hours")

# Leave this for grading purposes
ax_3d = plt.gca()

In [98]:
grader.check("q3d")

q3d results: All test cases passed!

##### Why do you think this happens? Are there more calls in the day or night? What is the least popular time? Most? (Answer Below)

### Question 4

It seems weekdays generally have slightly more calls than Saturday or Sunday, but the difference does not look significant.  

We can break down into some particular types of events to see their distribution. For example, let's make a bar plot for the CVLEGEND "ROBBERY". Which day is the peak for "ROBBERY"?

#### Question 4a

Use `pandas` to create a vertical bar plot of the number of total robberies reported on each day of the week, again ordered by the days of the week starting with Sunday. Please use the provided variable `filtered` in your answer.

**Hint:** *This should be very similar to Question 3b*

<!--
BEGIN QUESTION
name: q4a
-->

In [109]:
filtered = calls[calls["CVLEGEND"] == "ROBBERY"]
ax_4a = filtered['Day'].value_counts()[days].plot(kind = 'bar')
ax_4a.set_title("Number of Total Robberies Reported on Each Day of the Week")
ax_4a.set_ylabel("Number of Total Robberies")
# Leave this for grading purposes
ax_4a = plt.gca()

In [110]:
grader.check("q4a")

q4a results: All test cases passed!

#### Question 4b

Do you observe anything interesting about the distribution of ROBBERY calls over a week? Type a 1-sentence answer in the cell below.

### Question 5

In the cell below, generate a boxplot which examines the hour of day of each crime broken down by the `CVLEGEND` value.  To construct this plot use the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

##### Looking at your plot, which crime type appears to have the largest interquartile range? Put your results into `answer5` as a string.


<!--
BEGIN QUESTION
name: q5
-->

In [114]:
answer5 = "SEX CRIME"
calls.boxplot(column = "Hour", by= "CVLEGEND", rot=90);

In [115]:
grader.check("q5")

q5 results: All test cases passed!

**Important**: To make sure the test cases run correctly, click `Kernel>Restart & Run All` and make sure all of the test cases are still passing. Doing so will submit your code for you. 

If your test cases are no longer passing after restarting, it's likely because you're missing a variable, or that modifications that you'd previously made to your Data Frame are no longer taking place (perhaps because you deleted a cell). 

You may submit this assignment as many times as you'd like before the deadline.

**You must restart and run all cells before submitting. Otherwise, you may pass test cases locally, but not on our servers. We will not entertain regrade requests of the form, “my code passed all of my local test cases, but failed the autograder”.**

## Congratulations

Congrats! You are finished with this assignment.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [116]:
grader.check_all()

q1a results: All test cases passed!

q1b results: All test cases passed!

q2 results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q3d results: All test cases passed!

q4a results: All test cases passed!

q5 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before     running the cell below, so that all images/graphs appear in the output. The cell below will generate     a zipfile for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)